In [1]:
# ------------------------------------------------------------------------
#
#  Problem
#          \nabla^{2}u + u^{2} = f ,  -1 < x,y < 1,
#
#  with boundary conditions
#
#           u = \sin(2\pi x)\sin(2\pi y)
#           f = u^{2}(1 - 2 (2\pi)^{2})
#
#           u(x,y) = 0 along all boundaries
#
#
# ------------------------------------------------------------------------

try: range = xrange
except: pass

import sys, petsc4py
petsc4py.init(sys.argv)

from petsc4py import PETSc
import numpy as np

In [2]:
class PoissonEq(object):

    def __init__(self, da, minCoord, maxCoord):
        assert da.getDim() == 2
        self.da = da
        self.localX = da.createLocalVec()
        self.minCoord = minCoord
        self.maxCoord = maxCoord
        self.G  = da.createGlobalVec()
        
    def formFunction(self, snes, X, F):
        
        # global indices ranges on proc
        (xs, xe),(ys,ye) = self.da.getRanges()
        
        # get the vectors, reshape for easy
        self.da.globalToLocal(X, self.localX)
        x = self.da.getVecArray(self.localX) # getVecArray reshapes vector
        f = self.da.getVecArray(F)
        rhs = self.da.getVecArray(self.G)
               
        # work out global size & spacing
        nx, ny = self.da.getSizes()
        dx = float(self.maxCoord[0]-self.minCoord[0])/(nx-1)
        dy = float(self.maxCoord[1]-self.minCoord[1])/(ny-1)
                
        for j in range(ys, ye):
            for i in range(xs, xe):
                u   = x[i,j] # center
                u_w = u_e = u_s = u_n = 0
                if i > 0:    u_w = x[i-1,j] # west
                if i < nx-1: u_e = x[i+1,j] # east
                if j > 0:    u_s = x[i,j-1] # south
                if j < ny-1: u_n = x[i,j+1] # noth

                # calculate FD 2nd derivatives
                u_xx = (u_e - 2*u + u_w)/(dx*dx)
                u_yy = (u_n - 2*u + u_s)/(dy*dy)

                # residual
                f[i,j] = (u_xx + u_yy) + u*u - rhs[i,j]
        
        # the 4 dirichlet walls
        if xs ==  0:  f[xs,  ys:ye] = x[xs,  ys:ye] - 0   # west wall
        if xe == nx:  f[xe-1,ys:ye] = x[xe-1,ys:ye] - 0.  # east
        if ys ==  0:  f[xs:xe,  ys] = x[xs:xe,  ys] - 0   # south
        if ye == ny:  f[xs:xe,ye-1] = x[xs:xe,ye-1] - 0.  # north

In [3]:
# init system and get command arguments
comm = PETSc.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()

OptDB = PETSc.Options()
y_max = OptDB.getReal('y_max',1.)
x_max = OptDB.getReal('x_max',1.)
x_0   = -1*x_max
y_0   = -1*y_max

In [4]:
# attach 
da = PETSc.DMDA().create(sizes=[26,26], dof=1, stencil_width=1, comm=comm)

da.setUniformCoordinates(xmin=x_0, xmax=x_max, ymin=y_0, ymax=y_max)
da.setFromOptions()
da.setUp()
minCoord = [x_0, y_0]
maxCoord = [x_max, y_max]

# create the poisson object, no really reason
ctx = PoissonEq(da, minCoord, maxCoord)
da.setAppCtx(ctx)

In [5]:
def u_sol(k,xyz):
    return np.sin(k*xyz[0])*np.sin(k*xyz[1])

# create snes object and attach dm
snes = PETSc.SNES().create()
snes.setDM(da)
snes.setFromOptions()

x = da.createGlobalVec()  # unknowns
F = x.copy()              # residual
s = x.copy()              # analytic solution

# set residual function
snes.setFunction(ctx.formFunction, F)

# define rhs and analytic solution
rhs = da.getVecArray(ctx.G)
sol = da.getVecArray(s)

nx, ny = da.getSizes()
dx = float(x_max-x_0)/(nx-1)
dy = float(y_max-y_0)/(ny-1)
k = 2*np.pi

(xs, xe),(ys,ye) = da.getRanges()
for j in range(ys, ye):
    for i in range(xs,xe):
        xyz = i*dx+x_0, j*dy+y_0
        rhs[i,j] = u_sol(k,xyz)*(u_sol(k,xyz) - 2.*k*k) # RHS term       
        sol[i,j] = u_sol(k,xyz)
        
x.set(0.5) # initial solution

In [6]:
snes.solve(None, x)
   
print("Iteration count: {}".format(snes.getIterationNumber()))
x.axpy(-1,s)
err_norm = x.norm()
print("Error norm - {}".format(err_norm))

if OptDB.getBool('plot', True):
    draw = PETSc.Viewer.DRAW(x.comm)
    OptDB['draw_pause'] = 4
    draw(x)

Iteration count: 4
Error norm - 0.271095835839
